In [ ]:
import numpy as np
import pandas as pd
import k3d

from ipywidgets import interact, Output
from ipywidgets.widgets import FloatSlider, IntSlider, Button
from ipywidgets.widgets import HBox, interactive_output
from IPython.display import display, clear_output


def distance(a, b):
    return np.sqrt(np.sum((a-b)**2, axis=1))

def nearest_neighbours(origin, data, n_count):
    data = np.array(data)
    data_df = pd.DataFrame(data, columns=['x','y','z'])
    data_df['d'] = distance(origin, data)
    data_df = data_df.sort_values(by='d')
    nn = data_df[['x','y','z']][:n_count]
    return nn.values, nn.index

def xyz(n, x, y, z):
    pos = np.array([x, y, z], dtype=np.float32)
    point.positions = pos
    nn = nearest_neighbours(pos, data, n)
    
    data_points.colors = np.zeros(data.shape[0], dtype=np.uint32)
    data_points.colors[nn[1]] = 0xff0000
    data_points.colors = data_points.colors

def on_button_clicked(_):
    with out:
        clear_output()
        collected_data.append(nearest_neighbours(np.array([x.value, y.value, z.value]), data, n.value)[0])
        collected_indices.append(nearest_neighbours(np.array([x.value, y.value, z.value]), data, n.value)[1])
        print('Extracted!')

In [ ]:
X,Y,Z = np.meshgrid(np.linspace(0,4,7), np.linspace(0,4,7), np.linspace(0,4,7))
data = np.vstack([np.ravel(X), np.ravel(Y), np.ravel(Z)]).T
data += np.random.uniform(-0.05, 0.05, data.shape)

In [ ]:
collected_data, collected_indices = [], []
origin = np.array([data[:,0].min(), data[:,1].min(), data[:,2].min()])
end = np.array([data[:,0].max(), data[:,1].max(), data[:,2].max()])

In [ ]:
# visualisation widget
plot = k3d.plot()
point = k3d.points(origin.astype(np.float32), point_size=0.05, color=0x00ff00)

data_points = k3d.points(data.astype(np.float32), point_size=0.04, color=0) 
origin_point = k3d.points(origin.astype(np.float32), point_size=0.04)
end_point = k3d.points(end.astype(np.float32), point_size=0.04)

plot += point + data_points + origin_point + end_point

In [ ]:
# widget interface
n = IntSlider(min=0, description='n')
x = FloatSlider(min=origin[0], max=end[0], step=0.01, description='x')
y = FloatSlider(min=origin[1], max=end[1],step=0.01, description='y')
z = FloatSlider(min=origin[2], max=end[2], step=0.01, description='z')
b = Button(description='Extract')
b.on_click(on_button_clicked)

ui = HBox([n, x, y, z, b])
out = interactive_output(xyz, {'n':n,'x':x,'y':y,'z':z})    

display(ui, out)
plot.display()

In [ ]:
np.save('collected_data', collected_data)

In [ ]:
loaded_data = np.load('collected_data.npy')